In [1]:
import plotly.express as px
from glob import glob
import os
import pandas as pd
import csv
from collections import defaultdict

In [2]:
# import plotly.io as pio
# pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'
color_discrete_map = {
    'dfiv': '#7149C6',
    'dfiv_original': '#0F6292',
    'dfiv_hetero1': '#FF8400',
    'dfiv_hetero1.5': '#379237',
    'dfiv_hetero0.5': '#FF6464'
}

In [18]:
DUMP_DIR = '../dumps'
dump = {'dfiv': 'dfiv',
        'dfiv_original': 'dfiv_original',
        'dfiv_hetero1': 'dfiv_hetero1',
        'dfiv_hetero1.8': 'dfiv_hetero1.8',
        'dfiv_hetero0.2': 'dfiv_hetero0.2'}

In [12]:
alldict = defaultdict(list)
for key, dir in dump.items():
    m = 0
    for subdir in glob(os.path.join(DUMP_DIR, dir, 'data_size*')):
        params = [param.split(':') for param in subdir.split('/')[-1].split('-')]
        
        if os.path.exists(os.path.join(subdir, 'result.csv')):
            with open (os.path.join(subdir, 'result.csv'), 'r') as f:
                reader = csv.reader(f)
                mse = [float(row[0]) for row in list(reader)]
                n = len(mse)
                m += n

            params = {param[0]: [float(param[1])] * n for param in params}
            subdict = {**params, 'mse': mse, 'model': [key] * n}
            alldict = {key: subdict[key] + alldict[key] for key in subdict.keys()}

In [21]:
df = pd.DataFrame(alldict)

In [67]:
fig = px.box(df, x='data_size', y='mse', color='model', log_y=True,
                facet_col='rho', facet_col_wrap=3,
                color_discrete_map = color_discrete_map)
fig.show()

In [92]:
df_avg = df.groupby(['data_size', 'rho', 'model']).mean()
df_avg = df_avg.reset_index()
# df_avg

In [91]:
fig = px.bar(df_avg, x='data_size', y='mse', color='model', barmode='group',
                facet_col='rho', facet_col_wrap=3,
                color_discrete_map = color_discrete_map)
fig.update_xaxes(type='category')
fig.show()